In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('C:\\Users\\Mubasshira\\Downloads\\marketing_campaign_dataset.csv')
df.head(2)

,Campaign_ID,Company,Campaign_Type,Target_Audience,Duration,Channel_Used,Conversion_Rate,Acquisition_Cost,ROI,Location,Language,Clicks,Impressions,Engagement_Score,Customer_Segment,Date
0,1,Innovate Industries,Email,Men 18-24,30 days,Google Ads,0.04,16174.0,6.29,Chicago,Spanish,506,1922,6,Health & Wellness,01-01-2021
1,2,NexGen Systems,Email,Women 35-44,60 days,Google Ads,0.12,11566.0,5.61,New York,German,116,7523,7,Fashionistas,02-01-2021


# One-Way ANOVA

In [3]:
#Does the average Conversion Rate differ across different Campaign Types?
from scipy.stats import levene
group1 = df[df['Campaign_Type'] == "Influencer"]['Conversion_Rate']
group2 = df[df['Campaign_Type'] == "Search"]['Conversion_Rate']
group3 = df[df['Campaign_Type'] == "Email"]['Conversion_Rate']
group4 = df[df['Campaign_Type'] == "Display"]['Conversion_Rate']
group5 = df[df['Campaign_Type'] == "Social Media"]['Conversion_Rate']

stat, pval = levene(group1, group2, group3, group4, group5)

print("Levene Statistic:", stat)
print("P-value:", pval)

if pval < 0.05:
    print("Variances are significantly different — assumption violated.")
else:
    print("Variances are equal — assumption met.")

Levene Statistic: 1.4735454142821123
P-value: 0.20719589144227887
Variances are equal — assumption met.


In [4]:
#H₀ : The average Conversion Rate is the same across all Campaign Types.
#H₁ : At least one Campaign Type has a different average Conversion Rate compared to the others.

from scipy.stats import f_oneway
grouped_data = [group['Conversion_Rate'].values for name, group in df.groupby('Campaign_Type')]
f_stat, pval = f_oneway(*grouped_data)

print("F Statistic:", f_stat)
print("P-value:", pval)

if pval < 0.05:
    print("At least one Campaign Type has a different average Conversion Rate compared to the others")
else:
    print("The average Conversion Rate is the same across all Campaign Types.")

print("--------------------------------------------------------------")
from statsmodels.stats.multicomp import pairwise_tukeyhsd
tukey_test = pairwise_tukeyhsd(df['Conversion_Rate'], df['Campaign_Type'], alpha=0.05)
print(tukey_test)

F Statistic: 0.8872620760594464
P-value: 0.4704618058592427
The average Conversion Rate is the same across all Campaign Types.
--------------------------------------------------------------
     Multiple Comparison of Means - Tukey HSD, FWER=0.05     
  group1      group2    meandiff p-adj   lower  upper  reject
-------------------------------------------------------------
   Display        Email  -0.0003  0.834 -0.0011 0.0005  False
   Display   Influencer   0.0002 0.9341 -0.0006  0.001  False
   Display       Search  -0.0001 0.9993 -0.0009 0.0007  False
   Display Social Media      0.0 0.9999 -0.0007 0.0008  False
     Email   Influencer   0.0005 0.3535 -0.0003 0.0013  False
     Email       Search   0.0002 0.9273 -0.0006  0.001  False
     Email Social Media   0.0003 0.7487 -0.0004 0.0011  False
Influencer       Search  -0.0003 0.8438 -0.0011 0.0005  False
Influencer Social Media  -0.0002 0.9708  -0.001 0.0006  False
    Search Social Media   0.0001 0.9948 -0.0007 0.0009  False
----

In [5]:
#Is there a significant difference in ROI among different Customer Segments?
groups = [df[df['Customer_Segment'] == cs]['ROI'] for cs in df['Customer_Segment'].unique()]
stat, pval = levene(*groups)

print("Levene Statistic:", stat)
print("P-value:", pval)

if pval < 0.05:
    print("Variances are significantly different — assumption violated.")
else:
    print("Variances are equal — assumption met.")

print('----------------------------------------------------------------')
#H₀ : ROI among different Customer Segments is the same.
#H₁ : There is a significant difference in ROI among different Customer Segments.
f_stat, pval = f_oneway(*groups)
print("F Statistic:", f_stat)
print("P-value:", pval)

if pval < 0.05:
    print("There is a significant difference in ROI among different Customer Segments")
else:
    print("ROI among different Customer Segments is the same")

Levene Statistic: 0.3893075762532167
P-value: 0.8164574566132856
Variances are equal — assumption met.
----------------------------------------------------------------
F Statistic: 0.06325738303040701
P-value: 0.9926409718272996
ROI among different Customer Segments is the same


In [15]:
#Do Acquisition Costs vary significantly by Channel Used?
groups = [df[df['Channel_Used'] == cu]['Acquisition_Cost'] for cu in df['Channel_Used'].unique()]
stat, pval = levene(*groups)

print("Levene Statistic:", stat)
print("P-value:", pval)

if pval < 0.05:
    print("Variances are significantly different — assumption violated.")
else:
    print("Variances are equal — assumption met.")

print('--------------------------------------------------------')
#H₀ : The mean Acquisition Cost is the same across all Channels Used.
#H₁ : At least one Channel has a different mean Acquisition Cost compared to the others.
f_stat, pval = f_oneway(*groups)
print("F Statistic:", f_stat)
print("P-value:", pval)

if pval < 0.05:
    print("At least one Channel has a different mean Acquisition Cost compared to the others.")
else:
    print("The mean Acquisition Cost is the same across all Channels Used.")


Levene Statistic: 0.9939507995966484
P-value: 0.4195856130476713
Variances are equal — assumption met.
--------------------------------------------------------
F Statistic: 0.728070340697545
P-value: 0.6022658988253544
The mean Acquisition Cost is the same across all Channels Used.


In [33]:
#Is there a difference in Engagement Score between Language groups?
groups = [df[df['Language'] == l]['Engagement_Score'] for l in df['Language'].unique()]
stat, pval = levene(*groups)

print("Levene Statistic:", stat)
print("P-value:", pval)

if pval < 0.05:
    print("Variances are significantly different — assumption violated.")
else:
    print("Variances are equal — assumption met.")

print('--------------------------------------------------------')
#H₀ : There is no difference in Engagement Score between Language groups
#H₁ : At least one language group have a different engagement score compared to the others.
f_stat, pval = f_oneway(*groups)
print("F Statistic:", f_stat)
print("P-value:", pval)

if pval < 0.05:
    print("At least one language group have a different engagement score compared to the others")
else:
    print("There is no difference in Engagement Score between Language groups")


Levene Statistic: 1.1230460207376776
P-value: 0.34347857761854006
Variances are equal — assumption met.
--------------------------------------------------------
F Statistic: 1.1490581513155902
P-value: 0.3312922306835115
There is no difference in Engagement Score between Language groups


In [34]:
#I know that data is not normally distributed, just understanding how one way ANOVA works.
#If data is non normal we'll use Kruskal Wallis Test as applied below
from scipy.stats import kruskal
stat, pval = kruskal(*groups)

print("Kruskal-Wallis Statistic:", stat)
print("P-value:", pval)


Kruskal-Wallis Statistic: 4.605699891293212
P-value: 0.3301975292909653


# Two Way ANOVA

In [26]:
#Does Conversion Rate vary based on both Campaign Type and Channel Used?
print("Check for equal variance")
groups = [group['Conversion_Rate'].values for name, group in df.groupby(['Campaign_Type', 'Channel_Used'])]
stat, pval = levene(*groups)

print("Levene Statistic:", stat)
print("P-value:", pval)

if pval < 0.05:
    print("Variances are significantly different — assumption violated.")
else:
    print("Variances are equal — assumption met.")
print("------------------------------")
print("Check for normality")
from scipy.stats import shapiro

# Applying Shapiro test for each group of the dependent variable
for name, group in df.groupby(['Campaign_Type']):
    stat, pval = shapiro(group['Conversion_Rate'])
    print(f"{name} — W-stat: {stat:.4f}, p-value: {pval:.4f}")
    
for name, group in df.groupby(['Channel_Used']):
    stat, pval = shapiro(group['Conversion_Rate'])
    print(f"{name} — W-stat: {stat:.4f}, p-value: {pval:.4f}")

Check for equal variance
Levene Statistic: 0.9899054731403699
P-value: 0.4803933623774792
Variances are equal — assumption met.
------------------------------
Check for normality
('Display',) — W-stat: 0.9517, p-value: 0.0000
('Email',) — W-stat: 0.9516, p-value: 0.0000
('Influencer',) — W-stat: 0.9530, p-value: 0.0000
('Search',) — W-stat: 0.9525, p-value: 0.0000
('Social Media',) — W-stat: 0.9523, p-value: 0.0000
('Email',) — W-stat: 0.9530, p-value: 0.0000
('Facebook',) — W-stat: 0.9527, p-value: 0.0000
('Google Ads',) — W-stat: 0.9519, p-value: 0.0000
('Instagram',) — W-stat: 0.9523, p-value: 0.0000
('Website',) — W-stat: 0.9520, p-value: 0.0000
('YouTube',) — W-stat: 0.9518, p-value: 0.0000


In [23]:
#since p-value is < 0.05 the data is not normally distributed. Thus using GLM for hypothesis 
import statsmodels.api as sm
from statsmodels.formula.api import glm

model = glm('Conversion_Rate ~ C(Campaign_Type) + C(Channel_Used) + C(Campaign_Type):C(Channel_Used)', 
            data=df, 
            family=sm.families.Gaussian()).fit()

print(model.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:        Conversion_Rate   No. Observations:               200000
Model:                            GLM   Df Residuals:                   199970
Model Family:                Gaussian   Df Model:                           29
Link Function:               Identity   Scale:                       0.0016486
Method:                          IRLS   Log-Likelihood:             3.5701e+05
Date:                Thu, 17 Apr 2025   Deviance:                       329.67
Time:                        06:48:47   Pearson chi2:                     330.
No. Iterations:                     3   Pseudo R-squ. (CS):          7.993e-05
Covariance Type:            nonrobust                                         
                                                                     coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------

**"We tested whether different campaign types, channels, and their interactions significantly affect conversion rate. Although the model ran fine and assumptions of equal variance were met, the data is not normally distributed. Still, we used a GLM to handle this. However, the results show that no single campaign or channel had a statistically significant impact on conversion rates — all p-values > 0.05. Also, the model explains virtually none of the variability (Pseudo R² = 0.00008)."**

In [24]:
df.head(1)

,Campaign_ID,Company,Campaign_Type,Target_Audience,Duration,Channel_Used,Conversion_Rate,Acquisition_Cost,ROI,Location,Language,Clicks,Impressions,Engagement_Score,Customer_Segment,Date
0,1,Innovate Industries,Email,Men 18-24,30 days,Google Ads,0.04,16174.0,6.29,Chicago,Spanish,506,1922,6,Health & Wellness,01-01-2021


In [25]:
#Is there an interaction between Customer Segment and Language on ROI?
print("Check for equal variance")
groups = [group['ROI'].values for name, group in df.groupby(['Customer_Segment', 'Language'])]
stat, pval = levene(*groups)

print("Levene Statistic:", stat)
print("P-value:", pval)

if pval < 0.05:
    print("Variances are significantly different — assumption violated.")
else:
    print("Variances are equal — assumption met.")
print("------------------------------")
print("Check for normality")
for name, group in df.groupby(['Customer_Segment']):
    stat, pval = shapiro(group['ROI'])
    print(f"{name} — W-stat: {stat:.4f}, p-value: {pval:.4f}")

for name, group in df.groupby(['Language']):
    stat, pval = shapiro(group['ROI'])
    print(f"{name} — W-stat: {stat:.4f}, p-value: {pval:.4f}")

Check for equal variance
Levene Statistic: 0.7010594640005998
P-value: 0.8560023614601734
Variances are equal — assumption met.
------------------------------
Check for normality
('Fashionistas',) — W-stat: 0.9538, p-value: 0.0000
('Foodies',) — W-stat: 0.9548, p-value: 0.0000
('Health & Wellness',) — W-stat: 0.9545, p-value: 0.0000
('Outdoor Adventurers',) — W-stat: 0.9548, p-value: 0.0000
('Tech Enthusiasts',) — W-stat: 0.9548, p-value: 0.0000
('English',) — W-stat: 0.9546, p-value: 0.0000
('French',) — W-stat: 0.9552, p-value: 0.0000
('German',) — W-stat: 0.9546, p-value: 0.0000
('Mandarin',) — W-stat: 0.9544, p-value: 0.0000
('Spanish',) — W-stat: 0.9538, p-value: 0.0000


In [27]:
#since p-value is < 0.05 the data is not normally distributed. Thus using GLM for hypothesis 
import statsmodels.api as sm
from statsmodels.formula.api import glm

model = glm('ROI ~ C(Customer_Segment) + C(Language) + C(Customer_Segment):C(Language)', 
            data=df, 
            family=sm.families.Gaussian()).fit()

print(model.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                    ROI   No. Observations:               200000
Model:                            GLM   Df Residuals:                   199975
Model Family:                Gaussian   Df Model:                           24
Link Function:               Identity   Scale:                          3.0084
Method:                          IRLS   Log-Likelihood:            -3.9392e+05
Date:                Thu, 17 Apr 2025   Deviance:                   6.0161e+05
Time:                        06:51:59   Pearson chi2:                 6.02e+05
No. Iterations:                     3   Pseudo R-squ. (CS):          0.0001348
Covariance Type:            nonrobust                                         
                                                                         coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------

**The entire dataset is not normally distributed. Will still try two way Anova to examine and learn the features of ANOVA.**

In [32]:
#Do different Locations and Campaign Types influence Engagement Score?
import statsmodels.api as sm
from statsmodels.formula.api import ols
models = ols('Engagement_Score ~ C(Location) + C(Campaign_Type) + C(Location):C(Campaign_Type)', data=df).fit()
anova_table = sm.stats.anova_lm(models, typ=2)
print(anova_table)

                                    sum_sq        df         F    PR(>F)
C(Location)                   4.736524e+01       4.0  1.435040  0.219416
C(Campaign_Type)              1.400772e+01       4.0  0.424397  0.791156
C(Location):C(Campaign_Type)  1.709725e+02      16.0  1.295003  0.189538
Residual                      1.650104e+06  199975.0       NaN       NaN


Based on the p-values for Location, Campaign Type, and their interaction, none of these factors are statistically significant in predicting Engagement Score at the 5% significance level.

Therefore, these factors (individually or together) don't have a strong impact on Engagement Score in this model.